### 🛠️ 1. Initialize notebook variables

Configures everything that's needed for deployment. 

👉 **Modify entries under _1) User-defined parameters_ and _3) Define the APIs and their operations and policies_**.

In [1]:
import utils
import time
from apimtypes import *

# 1) User-defined parameters (change these as needed)
rg_location = 'eastus2'
index       = 2
deployment  = INFRASTRUCTURE.SIMPLE_APIM
tags        = ['oauth-3rd-party', 'jwt', 'credential-manager', 'policy-fragment']       # ENTER DESCRIPTIVE TAG(S)
api_prefix  = 'oauth-3rd-party-'               # OPTIONAL: ENTER A PREFIX FOR THE APIS TO REDUCE COLLISION POTENTIAL WITH OTHER SAMPLES

# 2) Service-defined parameters (please do not change these)
rg_name = utils.get_infra_rg_name(deployment, index)
supported_infrastructures = [INFRASTRUCTURE.SIMPLE_APIM, INFRASTRUCTURE.AFD_APIM_PE, INFRASTRUCTURE.APIM_ACA]        # ENTER SUPPORTED INFRASTRUCTURES HERE, e.g., [INFRASTRUCTURE.AFD_APIM_PE, INFRASTRUCTURE.AFD_APIM_FE]
utils.validate_infrastructure(deployment, supported_infrastructures)
sample_folder = "oauth-3rd-party"

# Set up the signing key for the JWT policy
jwt_key_name = f'JwtSigningKey{int(time.time())}'
jwt_key_value, jwt_key_value_bytes_b64 = utils.generate_signing_key()
utils.print_val('JWT key value', jwt_key_value)                         # this value is used to create the signed JWT token for requests to APIM
utils.print_val('JWT key value (base64)', jwt_key_value_bytes_b64)      # this value is used in the APIM validate-jwt policy's issuer-signing-key attribute  

# 3) Set up the named values
nvs: List[NamedValue] = [
    NamedValue(jwt_key_name, jwt_key_value_bytes_b64, True),
    NamedValue('MarketingMemberRoleId', Role.HR_MEMBER)
]

# # 4) Set up the policy fragments
# pf_authx_hr_member_xml = utils.read_policy_xml('pf-authx-hr-member.xml', {
#     'jwt_signing_key': jwt_key_name,
#     'hr_member_role_id': 'HRMemberRoleId'
# }, sample_folder)

# pfs: List[PolicyFragment] = [
#     PolicyFragment('AuthX-HR-Member', pf_authx_hr_member_xml, 'Authenticates and authorizes HR members.')
# ]

# 6) Define the APIs and their operations and policies

# Policies
pol_artist_get_xml  = utils.read_policy_xml('artist_get.xml', sample_name = sample_folder)
pol_spotify_api_xml = utils.read_policy_xml('spotify_api.xml', sample_name = sample_folder)

# Spotify
spotify_artist_hremployees_get = GET_APIOperation('Gets the Artists by ID', pol_artist_get_xml)

# APIs Array
apis: List[API] = [
    API(f'{api_prefix}spotify', 'Spotify API', f'/{api_prefix}spotify', 'This is the API for interactions with the Spotify REST API', policyXml = pol_spotify_api_xml, operations = [spotify_artist_hremployees_get], tags = tags),
]

utils.print_ok('Notebook initialized')

👉🏽 Resource group name      : apim-infra-simple-apim-2 
👉🏽 JWT key value            : FGUqleuzKPziT8gGSvN7zxqpSZN5AiLHwFdnLP78sbal 
👉🏽 JWT key value (base64)   : RkdVcWxldXpLUHppVDhnR1N2Tjd6eHFwU1pONUFpTEh3RmRuTFA3OHNiYWw= 
📄 Reading policy XML from  : C:\Dev\Azure-Samples\Apim-Samples\samples\oauth-3rd-party\artist_get.xml
📄 Reading policy XML from  : C:\Dev\Azure-Samples\Apim-Samples\samples\oauth-3rd-party\spotify_api.xml

✅ Notebook initialized ⌚ 00:33:06.402214 


### 🚀 2. Create deployment using Bicep

Creates the bicep deployment into the previously-specified resource group. A bicep parameters file will be created prior to execution.

In [ ]:
import utils

# 1) Define the Bicep parameters with serialized APIs
bicep_parameters = {
    'apis': {'value': [api.to_dict() for api in apis]},
    'namedValues': {'value': [nv.to_dict() for nv in nvs]}
}

# 2) Infrastructure must be in place before samples can be layered on top
if not utils.does_resource_group_exist(rg_name):
    utils.print_error(f'The specified infrastructure resource group and its resources must exist first. Please check that the user-defined parameters above are correctly referencing an existing infrastructure. If it does not yet exist, run the desired infrastructure in the /infra/ folder first.')
    raise SystemExit(1)

# 3) Run the deployment using the utility function that handles working directory management
output = utils.create_bicep_deployment_group_for_sample('authX-pro', rg_name, rg_location, bicep_parameters)

# 4) Print a deployment summary, if successful; otherwise, exit with an error
if not output.success:
    raise SystemExit('Deployment failed')

if output.success and output.json_data:
    apim_gateway_url = output.get('apimResourceGatewayURL', 'APIM API Gateway URL')

utils.print_ok('Deployment completed')

⚙️ az group show --name apim-infra-simple-apim-2 
📁 Changed working directory to: c:\Dev\Azure-Samples\Apim-Samples\samples\authX-pro
⚙️ az group show --name apim-infra-simple-apim-2 
📝 Updated the policy XML in the bicep parameters file 'params.json'
⚙️ az deployment group create --name authX-pro --resource-group apim-infra-simple-apim-2 --template-file "c:\Dev\Azure-Samples\Apim-Samples\samples\authX-pro\main.bicep" --parameters "c:\Dev\Azure-Samples\Apim-Samples\samples\authX-pro\params.json" --query "properties.outputs" 


### ✅ 3. Verify API Request Success

Assert that the deployment was successful by making simple calls to APIM. 

❗️ If the infrastructure shields APIM and requires a different ingress (e.g. Azure Front Door), the request to the APIM gateway URl will fail by design. Obtain the Front Door endpoint hostname and try that instead.

In [ ]:
# TBD